## Implementing Tools with LLMs

*[Coding along with the Udemy online course [LLM Engineering: Master AI & Large Language Models](https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models/) by Ed Donner; GitHub repo can be found at [github.com/ed-donner/llm_engineering](https://github.com/ed-donner/llm_engineering)]*

## Tools Project: Airline AI Assistant

In [1]:
from openai import OpenAI
import pandas as pd
import json
from pprint import pprint
import gradio as gr

In [2]:
openai_api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]

# connect to openai
openai = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o-mini"
print("Don't be a fool and sent your api key to github")

Don't be a fool and sent your api key to github


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message

"You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so."

In [5]:
# chat function in the style gradio expects it
# NO streaming back results this time
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [6]:
# brief check of gradio
gr.ChatInterface(fn=chat).launch()

/Users/juergenkober/Library/Caches/pypoetry/virtualenvs/llm-engineering-E34XR2a1-py3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
## Now, What are Tools?

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

Asking Claude.ai, 

> <span style="color:green">__What are tools in AI calls via an API?__</span>

Tools in AI APIs are external functions or capabilities that AI models can use to perform specific tasks. Think of them like specialized add-ons that expand what the AI can do beyond just processing text.

Let me explain with some common examples:

1. Code Interpreter
- Allows the AI to execute and run code
- Useful for data analysis, calculations, and generating visualizations

2. Web Browsing
- Enables the AI to search and retrieve current information from the internet
- Can help provide up-to-date facts and references

3. File Handling
- Lets the AI read, write, and manipulate different file types
- Examples: reading PDFs, analyzing spreadsheets, processing images

4. API Integration
- Allows the AI to interact with other services
- For example: checking weather data, sending emails, or accessing databases

When making API calls with tools:
- The developer specifies which tools the AI can access
- Each tool has defined inputs and outputs
- The AI determines when to use specific tools based on the task
- Results from tools can be incorporated into the AI's responses

Would you like me to elaborate on any specific aspect of AI tools or how they're implemented?

In [8]:
# let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

# get the ticket price for a given city
def get_ticket_price(destination_city):
    # starting with print statement to see later when this function is called
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

get_ticket_price("tokyo")

Tool get_ticket_price called for tokyo


'$1400'

Asking Claude.ai once again, 

> <span style="color:green">__There's a particular dictionary structure that's required to describe a function that can be used as a tool. Can you give me an example of this dictionary structure and describe it?__</span>

The tool/function definition in LLM API calls typically follows a JSON-like dictionary structure that describes the function's properties. Here's an example:

```python
tool = {
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Get the current price of a stock by its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., AAPL, GOOGL)"
                },
                "currency": {
                    "type": "string",
                    "enum": ["USD", "EUR", "GBP"],
                    "description": "The currency for the stock price",
                    "default": "USD"
                }
            },
            "required": ["symbol"]
        }
    }
}
```

Let's break down the key components:

1. Top Level:
   - `type`: Specifies this is a function tool
   - `function`: Contains the function's specifications

2. Function Properties:
   - `name`: The function identifier
   - `description`: Explains what the function does
   - `parameters`: Defines the input schema

3. Parameters Structure:
   - `type`: Usually "object" for the parameters container
   - `properties`: Defines each parameter:
     - Data type (string, number, boolean, etc.)
     - Description
     - Optional constraints (enum, min/max, pattern, etc.)
     - Default values
   - `required`: List of mandatory parameters


In [9]:
# There's a particular dictionary structure that's required to describe our function:
# description in plain ol' english so the llm understands when to call this function
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
# including the function (which is one tool) in a list of tools:
tools = [{"type": "function", "function": price_function}]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

#### __Giving an LLM the Power to Use our Tool__

What we actually going to do here is giving the LLM the opportunity to inform us that it wants us to run the tool we're providing. We're going to rewrite the `chat` function to allow OpenAI "to call our tool".

In [14]:
# the handle_tool_call function that we're going to use in the following chat() function
# go through the chat function first to understand the why and what
# parameter: message from gpt-4o
def handle_tool_call(message):
    # first we need to unpack the message to find out which tools we need to call
    tool_call = message.tool_calls[0]
    # second we need to extract the arguments from the tool call and extract them
    arguments = json.loads(tool_call.function.arguments) # gets returned into the form of json
    city = arguments.get('destination_city') # looking for parameters in arguments dictionary
    price = get_ticket_price(city) # calling our very own simple function
    # building the response
    response = {
        "role": "tool", # a new role here: tool
        "content": json.dumps({"destination_city": city,"price": price}), # dict turned into a string
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [15]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    # now calling chat.completions.create with tools as a parameter
    # the llm is informed by the description that this is a function it cal call
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # what are we checking for here?
    # the llm is sending a response back and will include 'finish_reason' into it
    # it means to tell us, that it doesn't know the answer yet and wants us to call one of the available tools
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message # getting the message that requests to run a tool
        # next: unpack the message, figure out what it wants to do and do it
        # handle_tool_call() will unpack the message from gpt and call our tool
        response, city = handle_tool_call(message)
        # next: adding two more rows to our list of messages 
        messages.append(message) # 1st: the message that we got back from gpt-4o
        messages.append(response) # 2nd: our result of calling the function
        response = openai.chat.completions.create(model=MODEL, messages=messages) # sending it all back to llm
        # we don't send in tools a second time because we don't expect that it will needed
    
    return response.choices[0].message.content

In [16]:
gr.ChatInterface(fn=chat).launch()

/Users/juergenkober/Library/Caches/pypoetry/virtualenvs/llm-engineering-E34XR2a1-py3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Tool get_ticket_price called for Paris
Tool get_ticket_price called for Tokyo
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Tangier
